In [2]:
from sender import Mail
import smtplib
from email.message import EmailMessage  # just once should suffice
from email.mime.text import MIMEText

In [18]:
my_email = 'huy.hoquang@ninjavan.co'
your_password = 'oakw pwyn uirh mseq'

In [20]:
def email(subject, body, to, attachment=None):

    msg = EmailMessage()
    msg.set_content(body)
    msg['subject'] = subject
    msg['to'] = to

    user = "huy.hoquang@ninjavan.co"
    msg['from'] = user
    password = "oakw pwyn uirh mseq"

    if attachment is not None:
        with open(attachment, 'rb') as fp:
            msg.add_attachment(
                fp.read(),
                maintype='attachment',
                subtype='octet-stream',
                filename=attachment)

    with smtplib.SMTP("smtp.gmail.com", 587) as s:
        s.ehlo()
        s.starttls()
        s.login(user, password)
        s.send_message(msg)
        s.quit()

In [23]:
if __name__ == '__main__':
    # email("testing", "testing testing", "XXXXXXXXX", "logs.txt")
    email("testing", "testing testing", "XXXXXXXXX")

SMTPRecipientsRefused: {'XXXXXXXXX': (553, b'5.1.3 The recipient address <XXXXXXXXX> is not a valid RFC 5321 address. For\n5.1.3 more information, go to\n5.1.3  https://support.google.com/a/answer/3221692 and review RFC 5321\n5.1.3 specifications. d9443c01a7336-207945daa86sm67461655ad.61 - gsmtp')}

In [16]:
with smtplib.SMTP_SSL('smtp.gmail.com', 465) as connection:
    connection.login(user=my_email, password=your_password)
    connection.send_message(
        from_addr=my_email,
        to_addrs=my_email,
        msg="Subj"
    )

AttributeError: 'str' object has no attribute 'get_all'

In [25]:
sender = 'huy.hoquang@ninjavan.co'
recipient = 'quanghuyho06@gmail.com'

msg = EmailMessage()
msg.set_content('this is a test')
msg['From'] = sender
msg['To'] = recipient
msg['Subject'] = 'test email'

# with smtplib.SMTP('smtp.office365.com', 587) as server:
with smtplib.SMTP('smtp.gmail.com', 465) as server:
    server.ehlo()
    server.starttls()
    server.ehlo()
    server.ehlo()
    server.login(sender, "oakw pwyn uirh mseq", initial_response_ok=True) 
    server.ehlo()
    server.sendmail(sender, recipient, msg.as_string())
    print('Email sent!')
    server.close()

SMTPServerDisconnected: Connection unexpectedly closed

Message sent!


In [6]:
from bs4 import BeautifulSoup

# Opening the html file 
HTMLFile = open("D:/NJV/project/send_email/teamplate_html.html", "r") 
  
# Reading the file 
index = HTMLFile.read() 

# Parse = BeautifulSoup(index, 'lxml') 
Parse = BeautifulSoup(index, 'html.parser') 

In [7]:
Parse

<!DOCTYPE html>
<html xmlns="http://www.w3.org/1999/xhtml" xmlns:o="urn:schemas-microsoft-com:office:office" xmlns:v="urn:schemas-microsoft-com:vml">
<head>
<title>
</title>
<!--[if !mso]><!-- -->
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<!--<![endif]-->
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<style type="text/css">
      #outlook a {
        padding: 0;
      }

      .ReadMsgBody {
        width: 100%;
      }

      .ExternalClass {
        width: 100%;
      }

      .ExternalClass * {
        line-height: 100%;
      }

      body {
        margin: 0;
        padding: 0;
        -webkit-text-size-adjust: 100%;
        -ms-text-size-adjust: 100%;
      }

      table,
      td {
        border-collapse: collapse;
        mso-table-lspace: 0pt;
        mso-table-rspace: 0pt;
      }

      img {
        border: 0;
        height: auto;
        line-height: 100%;
        

In [11]:
import os
root = os.path.abspath(os.getcwd())
# os.path.join(root, "main")
root

'd:\\NJV\\project\\send_email'

In [14]:
root = os.getcwd()
os.chdir(root)

with open('teamplate_html.html', mode="r", encoding="utf8") as template:
        index = template.read()
        body2 = BeautifulSoup(index, 'html.parser')

body2


<!DOCTYPE html>

<html xmlns="http://www.w3.org/1999/xhtml" xmlns:o="urn:schemas-microsoft-com:office:office" xmlns:v="urn:schemas-microsoft-com:vml">
<head>
<title>
</title>
<!--[if !mso]><!-- -->
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<!--<![endif]-->
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<style type="text/css">
      #outlook a {
        padding: 0;
      }

      .ReadMsgBody {
        width: 100%;
      }

      .ExternalClass {
        width: 100%;
      }

      .ExternalClass * {
        line-height: 100%;
      }

      body {
        margin: 0;
        padding: 0;
        -webkit-text-size-adjust: 100%;
        -ms-text-size-adjust: 100%;
      }

      table,
      td {
        border-collapse: collapse;
        mso-table-lspace: 0pt;
        mso-table-rspace: 0pt;
      }

      img {
        border: 0;
        height: auto;
        line-height: 100%;
       